In [2]:
import git
import subprocess
import pandas as pd
from pandas import json_normalize
import json
from collections import defaultdict
import csv
import os

In [48]:
test_repository_path = "E:/IUT/Seventh-Semister/SMP/Software Metrics/repositories/iosched"
base_dir = "E:/IUT/Seventh-Semister/SMP/Software Metrics/refactoring-productivity-analysis"

In [49]:
repo = git.Repo(test_repository_path)
commit_shas = [commit.hexsha for commit in repo.iter_commits()]
commit_shas[:10]

['738e1e008096fad5f36612325275e80c33dbe436',
 '3e4719b7ee24def7d6330dd2f736539d017da05f',
 '79ffcfef9fdbcf749dc09bba3dc504c240758be7',
 '117d1a98800696154d2b0b11389b743f0f9b3a9d',
 'e489d895d97c177f2adc904b113b46d03b439d93',
 'e1c07c6aa95968c6bab74143dc9ceaaafb43685f',
 '1ffc1a8c60ee2a3376e8882022d573d2e417c036',
 'b3b457292fffd077fd9a8a363ab14533b2e3c143',
 'aa45aea51c630a040f3e782f0c47d895861597f7',
 'e8489355ff88cea6d875c8cdae219160d1602281']

In [50]:
refactorig_miner_test_command = [f"{base_dir}/binaries/bin/RefactoringMiner.bat", "-h"]
refactorig_miner_test_command_mining = [f"{base_dir}/binaries/bin/RefactoringMiner.bat", 
                                        "-a",test_repository_path,"-json", f"{base_dir}/Dataset/testDataAllBranch.json"]

### Test Run of Refactoring miner

In [51]:
executor_process = subprocess.Popen(refactorig_miner_test_command_mining, stdout=subprocess.PIPE, shell=True, text=True)
output_text, _ = executor_process.communicate()
output_text

'Total count: [Commits: 3421, Errors: 0, Refactorings: 5228]\n'

In [52]:
# Specify the path to your JSON file
json_file_path = f"{base_dir}/Dataset/testDataAllBranch.json"
csv_file_path = f"{base_dir}/Dataset/refactoringDataset.csv"

In [53]:
file_empty = not os.path.isfile(csv_file_path) or os.path.getsize(csv_file_path) == 0

with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

commit_refactoring_types = defaultdict(set)

In [54]:
for commit in data['commits']:
    url = commit['url']

    refactoring_types = set()

    for refactoring in commit['refactorings']:
        refactoring_type = refactoring['type']

        refactoring_types.add(refactoring_type)

    commit_refactoring_types[url] = refactoring_types

In [55]:
csv_data = []
csv_headers = ['url', 'Extract Method', 'Move Field', 'Inline Method', 'Move Method', 'Pull Up Method', 'Pull Up Field',
               'Extract Superclass', 'Extract Interface', 'Push Down Method', 'Push Down Field', 'total refactoring types']

for commit in data['commits']:
    url = commit['url']
    refactoring_types = commit_refactoring_types[url]

    row_data = [url]

    for refactoring_type in csv_headers[1:-1]:
        row_data.append(1 if refactoring_type in refactoring_types else 0)

    row_data.append(len(refactoring_types))
    csv_data.append(row_data)

In [56]:

# Check if the CSV file is empty
file_empty = not os.path.isfile(csv_file_path) or os.path.getsize(csv_file_path) == 0

# Load JSON data from file
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

# Prepare data structures to store refactoring counts
commit_refactoring_types = defaultdict(set)

# Process each commit in the JSON data
for commit in data['commits']:
    url = commit['url']

    # Initialize a set to store unique refactoring types for each commit
    refactoring_types = set()

    # Process each refactoring in the commit
    for refactoring in commit['refactorings']:
        refactoring_type = refactoring['type']

        # Add refactoring type to the set
        refactoring_types.add(refactoring_type)

    # Update the set of refactoring types for the specific commit URL
    commit_refactoring_types[url] = refactoring_types

# Prepare data for writing to CSV
csv_data = []
csv_headers = ['url', 'Extract Method', 'Move Field', 'Inline Method', 'Move Method', 'Pull Up Method', 'Pull Up Field',
               'Extract Superclass', 'Extract Interface', 'Push Down Method', 'Push Down Field', 'total refactoring types']


# Append data for each commit
for commit in data['commits']:
    url = commit['url']
    refactoring_types = commit_refactoring_types[url]

    # Create a row for the CSV
    row_data = [url]

    # Append counts for each refactoring type
    for refactoring_type in csv_headers[1:-1]:
        row_data.append(1 if refactoring_type in refactoring_types else 0)

    # Append the total number of unique refactoring types
    row_data.append(len(refactoring_types))

    csv_data.append(row_data)

# Write data to CSV file
with open(csv_file_path, 'a', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # Write header only if the file is empty
    if file_empty:
        writer.writerow(csv_headers)

    # Write data without duplicating URLs
    existing_urls = set()
    for row in csv_data:
        if row[0] not in existing_urls:
            writer.writerow(row)
            existing_urls.add(row[0])


## Filter data of our interest

In [21]:

# Read the CSV file into a pandas DataFrame
data = pd.read_csv(csv_file_path)

# Filter rows where 'total refactoring types' is not equal to 0
filtered_data = data[data['total refactoring types'] != 0]


# Save the filtered data to a new CSV file
filtered_data.to_csv(f"{base_dir}/Dataset/refactoringDatasetFiltered.csv", index=False)